In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
n_Trials=5

In [ ]:
df=pd.read_csv('../input/train-folds/train_folds.csv')

In [ ]:
import sklearn
from sklearn.ensemble import RandomForestClassifier
import xgboost as xg
import joblib
import lightgbm as lgb
import optuna
from sklearn.tree import DecisionTreeClassifier

In [ ]:
def objective(trial):

    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    
    roc_auc_score=0
    clf = lgb.LGBMClassifier(**param)  
    for fold in range(1):
        
        df_train=df[df.kfold!=fold]
        df_test=df[df.kfold==fold]

        x_train=df_train.drop(['target','id','kfold'],axis=1).values
        y_train=df_train.target.values

        x_test=df_test.drop(['target','id','kfold'],axis=1).values
        y_test=df_test.target.values

    
        clf.fit(x_train,y_train)

        y_pred = clf.predict_proba(x_test)[:,1]
        roc_auc_score = roc_auc_score+sklearn.metrics.roc_auc_score(y_test,y_pred)
    
    return roc_auc_score

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=n_Trials)

In [ ]:
study.best_params

In [ ]:
n_folds=len(df['kfold'].value_counts())
print(n_folds)

In [ ]:
def run_folds(df,fold):
    df_train=df[df.kfold!=fold]
    df_test=df[df.kfold==fold]
    
    x_train=df_train.drop(['target','id','kfold'],axis=1).values
    y_train=df_train.target.values
    
    x_test=df_test.drop(['target','id','kfold'],axis=1).values
    y_test=df_test.target.values
    
    param=study.best_params
    param["objective"] = "binary"
    param["metric"]= "binary_logloss"
    param["verbosity"] = -1
    param["boosting_type"]= "gbdt"
    
    
    clf= lgb.LGBMClassifier(**param)
    clf.fit(x_train,y_train)
    
    y_pred = clf.predict_proba(x_test)[:,1]
    roc_auc_score = sklearn.metrics.roc_auc_score(y_test,y_pred)
    print(f"Fold={fold}, roc_auc_score={roc_auc_score}")
    
    File_name = 'model_lgbmclf' + str(fold)
    joblib.dump(
    clf,File_name)
    
    df_test.loc[:,"lgbmclf_pred"]=y_pred
    
    return df_test[['id','target','kfold','lgbmclf_pred']]


dfs=[]
    
for i in range(n_folds):
    temp_df=run_folds(df,i)
    dfs.append(temp_df)
df_pred=pd.concat(dfs)

print(df_pred.shape)

In [ ]:
df_pred.to_csv("lgbm_clf_predictions.csv",index=False)

In [ ]:
df_pred.head()

In [ ]:
df.head()